In [41]:
import os
import math
import matplotlib.image as mpimg
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np

from sklearn.model_selection import train_test_split
from keras.models import Sequential, model_from_json
from keras.layers import Dense, Conv2D, Flatten, MaxPooling2D, ZeroPadding2D, Convolution2D, Dropout
from keras.preprocessing.image import ImageDataGenerator, img_to_array
from keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint, TensorBoard
from sklearn.metrics import f1_score
from PIL import Image

from constant_values import *
from img_preprocessing import *
from LIDAR_model import *
from predictions import *

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


### Load full image dataset and augmentation done

In [42]:
TRAINING_SIZE = 1000
seed = np.random.randint(0, 10000)

In [43]:
tr_imgs = data_augmentation("../Data/training/images/", seed)      #100 images --> 2400 images
gt_imgs = data_augmentation("../Data/training/groundtruth/", seed) #100 images --> 2400 images

Image 100/100 is being loaded ... Shuffle data ...
Image 100/100 is being loaded ... Shuffle data ...


In [35]:
tr_imgs2 = tr_imgs[:TRAINING_SIZE, :, :, :]
gt_imgs2 = gt_imgs[:TRAINING_SIZE, :, :, :]

In [36]:
# Extract each images into patches (numpy arrays).
img = extract_data(tr_imgs2)  #shape: ((400/16)^2 * TRAINING_SIZE, 16, 16, 3)
gt = extract_labels(gt_imgs2) #shape: ((400/16)^2 * TRAINING_SIZE, 2)

In [37]:
TRAINING_SIZE

1000

In [11]:
#Test model simple

# Create model
# https://keras.io/models/sequential/
#mod = Sequential()

# Add a layer
# https://keras.io/layers/about-keras-layers/
# https://keras.io/activations/
#mod.add(Conv2D(64, kernel_size=IMG_PATCH_SIZE, activation='relu',input_shape=(IMG_PATCH_SIZE, IMG_PATCH_SIZE, NUM_CHANNELS)))
#mod.add(Flatten())
#mod.add(Dense(2, activation='softmax'))
        
        
# https://keras.io/optimizers/
# https://keras.io/losses/
#mod.compile(optimizer='sgd', loss='mean_squared_error', metrics=['accuracy']) #f1 = 0.43
#mod.compile(optimizer='adam',loss='categorical_crossentropy', metrics=['accuracy']) #f1 = 0.43

In [38]:
model = model_Lidar((16, 16, 3))
model.compile(optimizer='adam',loss='categorical_crossentropy', metrics=['accuracy'])

In [40]:
TEST_SIZE = 0.2 # To be studied

# Step 0: Shuffle samples
np.random.seed(0)
np.random.shuffle(img)
np.random.seed(0)
np.random.shuffle(gt)

# Step 1: Split into validation and training set     
Xtrain, Xtest, gt_train, gt_test = train_test_split(img, gt, test_size=TEST_SIZE, random_state=1)

# Step 2: Give weights to classes ?
c_weight = {1: 3., 
            0: 1.}

# Step 3: Generate Generators
train_datagenerator = ImageDataGenerator(horizontal_flip=True, vertical_flip=True)
test_datagenerator  = ImageDataGenerator()

train_generator = train_datagenerator.flow(Xtrain, gt_train, batch_size=BATCH_SIZE)
test_generator  = test_datagenerator.flow(Xtest, gt_test, batch_size=BATCH_SIZE)

# Step 4: Early stop
early_stop_callback = EarlyStopping(monitor='val_acc', min_delta=0, patience=15, verbose=0, mode='max', restore_best_weights=True)

# Step 5: Training
model.fit_generator(train_generator,
            validation_data=test_generator,
            steps_per_epoch=math.ceil(TRAINING_SIZE / BATCH_SIZE), #len(train_generator)/16,
            epochs=NUM_EPOCHS,
            callbacks = [early_stop_callback],
            class_weight=c_weight,
            validation_steps=math.ceil(len(test_generator)/BATCH_SIZE))

Epoch 1/100
63/63 [==============================] - 1243s 20s/step - loss: 0.9631 - acc: 0.8006 - val_loss: 0.6693 - val_acc: 0.8074
Epoch 2/100
63/63 [==============================] - 1121s 18s/step - loss: 0.9352 - acc: 0.8095 - val_loss: 0.5800 - val_acc: 0.8087
Epoch 3/100
63/63 [==============================] - 1153s 18s/step - loss: 0.9520 - acc: 0.8065 - val_loss: 0.6302 - val_acc: 0.8139
Epoch 4/100
63/63 [==============================] - 1199s 19s/step - loss: 1.0074 - acc: 0.7272 - val_loss: 0.6682 - val_acc: 0.8120
Epoch 5/100
62/63 [============================>.] - ETA: 4s - loss: 0.9425 - acc: 0.8075

KeyboardInterrupt: 

### Load model from Json 

In [16]:
from keras.models import model_from_json
json_file = open('simple_model.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
model = model_from_json(loaded_model_json)

In [25]:
train_data_filename = "../Data/training/images/"

prediction_training_dir = "predictions_training_lidar_model_LEO/"
if not os.path.isdir(prediction_training_dir):
    os.mkdir(prediction_training_dir)

for i in range(1, TRAINING_SIZE+1):
    #pimg = get_prediction_with_groundtruth(tr_imgs2[i], model)
    #Image.fromarray(pimg).save(prediction_training_dir + "prediction_" + str(i) + ".png")
    oimg = get_prediction_with_overlay(tr_imgs2[i], model)
    oimg.save(prediction_training_dir + "overlay_" + str(i) + ".png")
    sys.stdout.write("\rImage {}/{} is being processed".format(i,TRAINING_SIZE+1))
    sys.stdout.flush()

1
11
avant
625/625 [==============================] - 68s 109ms/step
apres
12
2
3
avant
625/625 [==============================] - 69s 111ms/step
apres
4
5
Image 1/101 is being processed1
11
avant
625/625 [==============================] - 73s 117ms/step
apres
12
2
3
avant
625/625 [==============================] - 71s 114ms/step
apres
4
5
Image 2/101 is being processed1
11
avant
625/625 [==============================] - 72s 115ms/step
apres
12
2
3
avant
 64/625 [==>...........................] - ETA: 1:06

KeyboardInterrupt: 

### Save model to json

In [ ]:

# serialize model to JSON
OUTPUT_FILENAME = "simple_model"
model_json = model.to_json()
print("1")
with open(OUTPUT_FILENAME + ".json", "w") as json_file:
    json_file.write(model_json)
    
print("2")
# serialize weights to HDF5
model.save_weights(OUTPUT_FILENAME + ".h5")
print("Saved model to disk")

In [ ]:
validation_data_prediction = model.predict_classes(Xtest, verbose = 1)
print("3")
validation_label = []
for e in gt_test:
    print("4")
    if (e[0] == 0):
        validation_label.append(1)
    else:
        validation_label.append(0)
        
validation_label = np.array(validation_label)     
print("F1 score = "+str(f1_score(validation_data_prediction, validation_label, labels=['1'])))